# Logistic 回归


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1 数据预处理


In [8]:
data = pd.read_csv("/Users/zhaohaonan/北大资料/Coding/MachineLearning/Datasets/Iris.csv")

# 查看分类信息
display(data["Species"].value_counts())


# 将字符串转换为数字
data_logistic = data.copy()
data_logistic["Species"] = data_logistic["Species"].map(
    {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
)

display(data_logistic.head())

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


## 2 数据集划分


In [93]:
# 数据集打乱
shuffled_data = data_logistic.iloc[:100, :].sample(
    frac=1.0, random_state=1
)  # frac=1.0表示全部打乱

# 划分训练集和测试集
train_data = shuffled_data.iloc[:80, :]
test_data = shuffled_data.iloc[80:, :]

# 将训练集和测试集分为特征和标签
train_X = np.array(train_data.iloc[:, 1:5]).reshape(-1, 4)
train_Y = np.array(train_data.iloc[:, 5]).reshape(-1, 1)

test_X = np.array(test_data.iloc[:, 1:5]).reshape(-1, 4)
test_Y = np.array(test_data.iloc[:, 5]).reshape(-1, 1)

display(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(80, 4)

(80, 1)

(20, 4)

(20, 1)

## 3 梯度下降训练模型


In [121]:
# 定义训练参数
from tkinter import Y


ALPHA = 2.5  # 学习率
MAX_ITER = 1000000  # 最大迭代次数
ESP = 1e-7  # 梯度下降收敛阈值


# 定义sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# 定义损失函数
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


# 定义梯度下降函数
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]


# 定义预测函数
def predict(X, theta):
    h = sigmoid(np.dot(X, theta))
    return h.round()


# 定义状态输出函数
def echo(X, h, y, theta, step):
    print(
        "Epoch: {} \t Loss: {:10f} \t Accuracy: {:4f}".format(
            step, loss(h, y), (predict(X, theta) == y).mean()
        )
    )
    return


# 定义训练函数
def train(X, y, alpha=ALPHA, max_iter=MAX_ITER):
    theta = np.zeros((X.shape[1], 1))
    step = 0
    while step < max_iter:
        h = sigmoid(np.dot(X, theta))
        gradient = gradient_descent(X, h, y)

        if loss(h, y) < ESP:
            print("梯度下降已经收敛")
            break

        theta -= alpha * gradient

        step += 1

        if step % 100000 == 0:
            echo(X, h, y, theta, step)

    return theta


# 训练模型
theta = train(train_X, train_Y)
print(theta.ravel())

# 预测
predict_Y = predict(test_X, theta)

# 计算准确率
print("Predict Accuracy: {:7f}".format((predict_Y == test_Y).mean()))

/var/folders/z7/0bpm4x0d1l12by_8_w62qyd80000gn/T/ipykernel_4690/1118144350.py:17: RuntimeWarning: divide by zero encountered in log
  return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
/var/folders/z7/0bpm4x0d1l12by_8_w62qyd80000gn/T/ipykernel_4690/1118144350.py:17: RuntimeWarning: invalid value encountered in multiply
  return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


Epoch: 100000 	 Loss:   0.000005 	 Accuracy: 1.000000
Epoch: 200000 	 Loss:   0.000003 	 Accuracy: 1.000000
Epoch: 300000 	 Loss:   0.000002 	 Accuracy: 1.000000
Epoch: 400000 	 Loss:   0.000002 	 Accuracy: 1.000000
Epoch: 500000 	 Loss:   0.000001 	 Accuracy: 1.000000
Epoch: 600000 	 Loss:   0.000001 	 Accuracy: 1.000000
Epoch: 700000 	 Loss:   0.000001 	 Accuracy: 1.000000
Epoch: 800000 	 Loss:   0.000001 	 Accuracy: 1.000000
Epoch: 900000 	 Loss:   0.000001 	 Accuracy: 1.000000
Epoch: 1000000 	 Loss:   0.000001 	 Accuracy: 1.000000
[-2.57745237 -6.99155966 11.77197417  5.41975148]
Predict Accuracy: 1.000000


In [92]:
# 使用sklearn库对比手写模型得到的结果
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(penalty=None, fit_intercept=False)
sklearn_model.fit(train_X[:, 1:6], train_Y.ravel())
sklearn_predict_Y = sklearn_model.predict(test_X[:, 1:6])

# 打印 sklearn 模型的参数
print("Sklearn Model Coef: ", sklearn_model.coef_)
print("Sklearn Model Intercept: ", sklearn_model.intercept_)
print("Sklearn Model Classes: ", sklearn_model.classes_)
print("Sklearn Model Predict: ", sklearn_predict_Y)

# 计算损失函数
sklearn_loss = loss(
    sigmoid(np.dot(train_X[:, 1:6], sklearn_model.coef_.T) + sklearn_model.intercept_),
    train_Y,
)
print("Sklearn Model Loss: ", sklearn_loss)

# 计算准确率
print(
    "Sklearn Predict Accuracy: {:7f}".format(
        (sklearn_predict_Y == test_Y.ravel()).mean()
    )
)

Sklearn Model Coef:  [[-2.4759644  -7.25215035 11.72101698  5.548456  ]]
Sklearn Model Intercept:  [0.]
Sklearn Model Classes:  [0 1]
Sklearn Model Predict:  [0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 0]
Sklearn Model Loss:  6.397524371360649e-07
Sklearn Predict Accuracy: 1.000000
